#Make bookkeeping with Holograms


- work with Weakly_2022_09
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/06/16


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/atmospec 	setup
eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_09/conda/envs/lsst-scipipe-2.0.0/eups 	setup
rapid_analysis        LOCAL:/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/repos/w_2022_09/rapid_analysis 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g64fc59b30a+e1dba0a6f6 	w_latest w_2022_09 current setup


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [4]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [5]:
import lsst.daf.butler as dafButler

In [6]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [7]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs','group_name' ,'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [8]:
df_exposure.astype({"id":str,
                    "obs_id":str,
                    "day_obs": str,
                    "group_name":str,
                    "seq_num":'int32',
                    'type':str,
                    "target":str,
                    "filter":str,
                    "zenith_angle":'float',
                    "expos":'float',
                    "ra":'float',
                    "dec":'float',
                    "skyangle":'float'
                   }              
                  )

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle


In [9]:
#df_exposure = pd.DataFrame( {"id":'int64',
#                             "obs_id":str,
#                             "day_obs": str,
#                             "seq_num":'int32',
#                             "type":str,
#                             "target":str,
#                             "filter":str,
#                             "zenith_angle":'float32',
#                             "expos":'float32',
#                             "ra":'float32',
#                             "dec":'float32',
#                             "skyangle":'float32'})                  

In [10]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    
    df_exposure.loc[count] = [str(info.id), str(info.obs_id), str(info.day_obs), str(info.group_name),int(info.seq_num), str(info.observation_type), str(info.target_name), str(info.physical_filter), info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t group_name:          ",info.group_name)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [11]:
df_exposure.dtypes

id               object
obs_id           object
day_obs          object
group_name       object
seq_num           int64
type             object
target           object
filter           object
zenith_angle     object
expos           float64
ra               object
dec              object
skyangle         object
dtype: object

In [12]:
df_exposure

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2021021600208,AT_O_20210216_000208,20210216,2021-02-17T03:35:45.035,208,science,HD 50896,empty~empty,24.314073,2.0,103.384002,-23.973448,-180.0
1,2021021600163,AT_O_20210216_000163,20210216,2021-02-17T00:50:15.536,163,science,HD 50896,RG610~empty,14.580611,3.0,103.404029,-23.92939,-208.621712
2,2021021600070,AT_O_20210216_000070,20210216,2021-02-16T20:39:31.417,70,bias,azel_target,empty~empty,None,0.0,None,None,None
3,2021021600173,AT_O_20210216_000173,20210216,2021-02-17T01:21:19.215,173,science,HD 50896,RG610~empty,8.82931,2.0,103.395986,-23.927038,-208.621712
4,2021021600001,AT_O_20210216_000001,20210216,2021-02-16T19:50:21.993,1,bias,azel_target,empty~empty,None,0.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19070,2022060900838,AT_O_20220609_000838,20220609,2022-06-10T08:52:06.482,838,science,spec:HD185975,BG40~holo4_003,57.194196,30.0,307.708009,-87.50686,105.70201
19071,2022060900839,AT_O_20220609_000839,20220609,2022-06-10T08:52:06.482,839,science,spec:HD185975,FELH0600~holo4_003,57.196255,30.0,307.707897,-87.506868,105.701922
19072,2022060900840,AT_O_20220609_000840,20220609,2022-06-10T08:52:06.482,840,science,spec:HD185975,FELH0600~holo4_003,57.198265,30.0,307.708264,-87.506854,105.701541
19073,2022060900841,AT_O_20220609_000841,20220609,2022-06-10T08:52:06.482,841,science,spec:HD185975,FELH0600~holo4_003,57.200339,30.0,307.708109,-87.506873,105.701644


## Selection of science

In [13]:
df_science = df_exposure[df_exposure.type == 'science']

In [14]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_4066/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [15]:
df_science.tail(60)

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
18998,2022060900766,AT_O_20220609_000766,20220609,2022-06-10T07:10:46.665,766,science,spec:HD205905,empty~ronchi170lpmm,19.548686,30.0,324.794559,-27.258511,346.341748
18999,2022060900767,AT_O_20220609_000767,20220609,2022-06-10T07:10:46.665,767,science,spec:HD205905,empty~ronchi170lpmm,19.393817,30.0,324.794577,-27.258501,346.341711
19000,2022060900768,AT_O_20220609_000768,20220609,2022-06-10T07:10:46.665,768,science,spec:HD205905,empty~ronchi170lpmm,19.238899,30.0,324.79452,-27.258501,346.341765
19001,2022060900769,AT_O_20220609_000769,20220609,2022-06-10T07:10:46.665,769,science,spec:HD205905,empty~ronchi170lpmm,19.084132,30.0,324.794609,-27.258488,346.341744
19002,2022060900770,AT_O_20220609_000770,20220609,2022-06-10T07:10:46.665,770,science,spec:HD205905,BG40~ronchi170lpmm,18.918483,30.0,324.794587,-27.258509,346.341739
19003,2022060900771,AT_O_20220609_000771,20220609,2022-06-10T07:10:46.665,771,science,spec:HD205905,BG40~ronchi170lpmm,18.763642,30.0,324.794536,-27.258496,346.341758
19004,2022060900772,AT_O_20220609_000772,20220609,2022-06-10T07:10:46.665,772,science,spec:HD205905,BG40~ronchi170lpmm,18.608877,30.0,324.794553,-27.258511,346.337373
19005,2022060900773,AT_O_20220609_000773,20220609,2022-06-10T07:10:46.665,773,science,spec:HD205905,BG40~ronchi170lpmm,18.454103,30.0,324.79453,-27.258523,346.341696
19006,2022060900774,AT_O_20220609_000774,20220609,2022-06-10T07:10:46.665,774,science,spec:HD205905,FELH0600~ronchi170lpmm,18.29219,30.0,324.794535,-27.258508,346.341701
19007,2022060900775,AT_O_20220609_000775,20220609,2022-06-10T07:10:46.665,775,science,spec:HD205905,FELH0600~ronchi170lpmm,18.137542,30.0,324.79459,-27.258507,346.341849


# Generate Book Keeping files

In [16]:
writer = pd.ExcelWriter('BookKeepingAuxtelHologram_raw.xlsx', engine='xlsxwriter')
current_page = 0

In [17]:
ListOfDates = df_science.day_obs.unique()

In [18]:
ListOfDates

array(['20210216', '20210217', '20210218', '20210311', '20210608',
       '20210609', '20210610', '20210706', '20210707', '20210708',
       '20210907', '20210908', '20210909', '20211005', '20211006',
       '20211102', '20211103', '20211104', '20220215', '20220216',
       '20220217', '20220316', '20220317', '20220503', '20220504',
       '20220524', '20220607', '20220608', '20220609'], dtype=object)

In [19]:
for date_sel in ListOfDates:
    
    # get for that date
    df_science_selected = df_science[df_science.day_obs == date_sel ]
    N = len(df_science_selected)
    
    list_of_filters = df_science_selected['filter'].unique()
    
    selected_filters = []
    # Select hologram
    for filt in list_of_filters:
        #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
        flag_sel = (filt.find('holo4') != -1) 
        #flag_sel = (filt.find('ronchi170') != -1) 
        if flag_sel:
            selected_filters.append(filt) 
    Nfilt = len(selected_filters)
    # sort by filter name
    if(Nfilt>0):
        selected_filters = np.array(selected_filters)
        df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
        df_science_selected.to_excel(writer, sheet_name=str(date_sel))
        


/tmp/ipykernel_4066/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
/tmp/ipykernel_4066/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
/tmp/ipykernel_4066/4092091253.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science_selected.sort_va

In [20]:
writer.save()